# Lesson A2: Smart Pointers & Interior Mutability

**Duration**: 120-135 minutes  
**Stage**: Advanced (Mastery)  
**Prerequisites**: Lesson A1 (Lifetimes & Advanced Traits)

---

## 📋 What You'll Learn

This lesson explores Rust's smart pointer types and interior mutability patterns. You'll learn when and how to use Box, Rc, Arc, RefCell, and other smart pointers to manage memory and enable advanced ownership patterns.

**Why this matters**: Smart pointers enable patterns that would be impossible with basic references alone - from recursive data structures to shared ownership. Understanding them is essential for building complex applications like graph databases, UI frameworks, and concurrent systems.

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Use Box<T> for heap allocation and recursive types
2. Work with Rc<T> and Arc<T> for reference counting
3. Implement interior mutability with RefCell<T> and Mutex<T>
4. Understand weak references and cycle prevention
5. Create custom smart pointers

---

## 📋 Prerequisite Review

**Quick Check**: From our previous lessons:

1. What are associated types in traits?
2. How do trait objects enable dynamic dispatch?
3. What are higher-ranked trait bounds?
4. What's the difference between Fn, FnMut, and FnOnce?

**Answers**: 1) Types associated with trait implementations, 2) Runtime polymorphism via vtables, 3) Universal quantification over lifetimes, 4) Different levels of closure capture and mutability

---

## 🧠 Key Concepts

### Smart Pointer Types

**Box<T>**: Heap allocation, single ownership
**Rc<T>**: Reference counting, shared ownership (single-threaded)
**Arc<T>**: Atomic reference counting, shared ownership (multi-threaded)
**RefCell<T>**: Interior mutability with runtime borrow checking
**Mutex<T>**: Thread-safe interior mutability

### Memory Management Patterns

- **RAII**: Resource Acquisition Is Initialization
- **Reference Cycles**: Potential memory leaks with Rc<T>
- **Weak References**: Breaking cycles with Weak<T>
- **Interior Mutability**: Mutating through shared references

---

## 🔬 Live Code Exploration

### Box<T> for Heap Allocation

In [ ]:
// Box<T> for heap allocation and recursive types

// Recursive data structure using Box
#[derive(Debug)]
enum List<T> {
    Cons(T, Box<List<T>>),
    Nil,
}

impl<T> List<T> {
    fn new() -> Self {
        List::Nil
    }
    
    fn prepend(self, elem: T) -> Self {
        List::Cons(elem, Box::new(self))
    }
    
    fn len(&self) -> usize {
        match self {
            List::Cons(_, tail) => 1 + tail.len(),
            List::Nil => 0,
        }
    }
    
    fn stringify(&self) -> String
    where
        T: std::fmt::Display,
    {
        match self {
            List::Cons(head, tail) => {
                if tail.len() > 0 {
                    format!("{}, {}", head, tail.stringify())
                } else {
                    format!("{}", head)
                }
            }
            List::Nil => String::new(),
        }
    }
}

// Binary tree using Box
#[derive(Debug)]
struct TreeNode<T> {
    value: T,
    left: Option<Box<TreeNode<T>>>,
    right: Option<Box<TreeNode<T>>>,
}

impl<T> TreeNode<T> {
    fn new(value: T) -> Self {
        TreeNode {
            value,
            left: None,
            right: None,
        }
    }
    
    fn insert_left(&mut self, value: T) {
        self.left = Some(Box::new(TreeNode::new(value)));
    }
    
    fn insert_right(&mut self, value: T) {
        self.right = Some(Box::new(TreeNode::new(value)));
    }
    
    fn height(&self) -> usize {
        let left_height = self.left.as_ref().map_or(0, |node| node.height());
        let right_height = self.right.as_ref().map_or(0, |node| node.height());
        1 + left_height.max(right_height)
    }
}

impl<T: std::fmt::Display> TreeNode<T> {
    fn print_inorder(&self) {
        if let Some(left) = &self.left {
            left.print_inorder();
        }
        print!("{} ", self.value);
        if let Some(right) = &self.right {
            right.print_inorder();
        }
    }
}

// Large data structure that benefits from heap allocation
struct LargeData {
    data: [u8; 1024], // 1KB of data
    id: u32,
}

impl LargeData {
    fn new(id: u32) -> Box<Self> {
        Box::new(LargeData {
            data: [0; 1024],
            id,
        })
    }
    
    fn process(&self) -> String {
        format!("Processing large data with ID: {}", self.id)
    }
}

fn box_demo() {
    println!("=== Box<T> for Heap Allocation ===");
    
    // Linked list example
    let list = List::new()
        .prepend(1)
        .prepend(2)
        .prepend(3);
    
    println!("List: [{}]", list.stringify());
    println!("List length: {}", list.len());
    
    // Binary tree example
    let mut root = TreeNode::new("root");
    root.insert_left("left");
    root.insert_right("right");
    
    if let Some(left) = &mut root.left {
        left.insert_left("left-left");
        left.insert_right("left-right");
    }
    
    println!("\nTree structure: {:?}", root);
    println!("Tree height: {}", root.height());
    print!("Inorder traversal: ");
    root.print_inorder();
    println!();
    
    // Large data on heap
    let large_data1 = LargeData::new(1);
    let large_data2 = LargeData::new(2);
    
    println!("\n{}", large_data1.process());
    println!("{}", large_data2.process());
    
    // Box allows moving large data efficiently
    let data_vec = vec![large_data1, large_data2];
    println!("Moved {} large data items to vector", data_vec.len());
}

box_demo();

### Rc<T> and Reference Counting

In [ ]:
// Rc<T> for reference counting and shared ownership

use std::rc::{Rc, Weak};
use std::cell::RefCell;

// Shared data structure
#[derive(Debug)]
struct SharedData {
    value: i32,
    name: String,
}

impl SharedData {
    fn new(value: i32, name: String) -> Self {
        SharedData { value, name }
    }
    
    fn display(&self) {
        println!("SharedData: {} = {}", self.name, self.value);
    }
}

// Node in a graph with shared references
#[derive(Debug)]
struct GraphNode {
    id: u32,
    data: String,
    neighbors: Vec<Rc<RefCell<GraphNode>>>,
    parent: Option<Weak<RefCell<GraphNode>>>,
}

impl GraphNode {
    fn new(id: u32, data: String) -> Rc<RefCell<Self>> {
        Rc::new(RefCell::new(GraphNode {
            id,
            data,
            neighbors: Vec::new(),
            parent: None,
        }))
    }
    
    fn add_neighbor(node: &Rc<RefCell<Self>>, neighbor: &Rc<RefCell<Self>>) {
        node.borrow_mut().neighbors.push(Rc::clone(neighbor));
        neighbor.borrow_mut().parent = Some(Rc::downgrade(node));
    }
    
    fn display(&self) {
        println!("Node {}: {} (neighbors: {})", 
                self.id, self.data, self.neighbors.len());
    }
    
    fn display_neighbors(&self) {
        println!("Node {} neighbors:", self.id);
        for neighbor in &self.neighbors {
            let neighbor_ref = neighbor.borrow();
            println!("  - Node {}: {}", neighbor_ref.id, neighbor_ref.data);
        }
    }
}

// Tree structure with parent references using Weak
#[derive(Debug)]
struct TreeNodeRc {
    value: String,
    children: Vec<Rc<RefCell<TreeNodeRc>>>,
    parent: Option<Weak<RefCell<TreeNodeRc>>>,
}

impl TreeNodeRc {
    fn new(value: String) -> Rc<RefCell<Self>> {
        Rc::new(RefCell::new(TreeNodeRc {
            value,
            children: Vec::new(),
            parent: None,
        }))
    }
    
    fn add_child(parent: &Rc<RefCell<Self>>, child_value: String) -> Rc<RefCell<Self>> {
        let child = TreeNodeRc::new(child_value);
        child.borrow_mut().parent = Some(Rc::downgrade(parent));
        parent.borrow_mut().children.push(Rc::clone(&child));
        child
    }
    
    fn print_tree(node: &Rc<RefCell<Self>>, depth: usize) {
        let indent = "  ".repeat(depth);
        let node_ref = node.borrow();
        println!("{}{}", indent, node_ref.value);
        
        for child in &node_ref.children {
            Self::print_tree(child, depth + 1);
        }
    }
    
    fn get_path_to_root(&self) -> Vec<String> {
        let mut path = vec![self.value.clone()];
        
        if let Some(parent_weak) = &self.parent {
            if let Some(parent) = parent_weak.upgrade() {
                let parent_path = parent.borrow().get_path_to_root();
                path.extend(parent_path);
            }
        }
        
        path
    }
}

fn rc_demo() {
    println!("\n=== Rc<T> and Reference Counting ===");
    
    // Basic Rc usage
    let shared_data = Rc::new(SharedData::new(42, "Answer".to_string()));
    
    println!("Initial reference count: {}", Rc::strong_count(&shared_data));
    
    {
        let data_ref1 = Rc::clone(&shared_data);
        let data_ref2 = Rc::clone(&shared_data);
        
        println!("Reference count with clones: {}", Rc::strong_count(&shared_data));
        
        data_ref1.display();
        data_ref2.display();
    } // data_ref1 and data_ref2 go out of scope
    
    println!("Reference count after scope: {}", Rc::strong_count(&shared_data));
    
    // Graph with shared nodes
    let node1 = GraphNode::new(1, "Node One".to_string());
    let node2 = GraphNode::new(2, "Node Two".to_string());
    let node3 = GraphNode::new(3, "Node Three".to_string());
    
    GraphNode::add_neighbor(&node1, &node2);
    GraphNode::add_neighbor(&node1, &node3);
    GraphNode::add_neighbor(&node2, &node3);
    
    println!("\nGraph structure:");
    node1.borrow().display();
    node1.borrow().display_neighbors();
    
    node2.borrow().display();
    node2.borrow().display_neighbors();
    
    // Tree with parent references
    let root = TreeNodeRc::new("Root".to_string());
    let child1 = TreeNodeRc::add_child(&root, "Child 1".to_string());
    let child2 = TreeNodeRc::add_child(&root, "Child 2".to_string());
    let grandchild1 = TreeNodeRc::add_child(&child1, "Grandchild 1".to_string());
    let grandchild2 = TreeNodeRc::add_child(&child1, "Grandchild 2".to_string());
    
    println!("\nTree structure:");
    TreeNodeRc::print_tree(&root, 0);
    
    // Path to root from grandchild
    let path = grandchild1.borrow().get_path_to_root();
    println!("\nPath from Grandchild 1 to root: {:?}", path);
    
    // Reference counts
    println!("\nReference counts:");
    println!("Root: {}", Rc::strong_count(&root));
    println!("Child 1: {}", Rc::strong_count(&child1));
    println!("Grandchild 1: {}", Rc::strong_count(&grandchild1));
    
    // Weak reference count
    println!("Root weak count: {}", Rc::weak_count(&root));
}

rc_demo();

In [ ]:
// RefCell<T> for interior mutability

use std::rc::Rc;
use std::cell::{RefCell, Cell};

// Mock object pattern using RefCell
trait Messenger {
    fn send(&self, msg: &str);
}

struct MockMessenger {
    sent_messages: RefCell<Vec<String>>,
}

impl MockMessenger {
    fn new() -> Self {
        MockMessenger {
            sent_messages: RefCell::new(Vec::new()),
        }
    }
    
    fn messages(&self) -> Vec<String> {
        self.sent_messages.borrow().clone()
    }
}

impl Messenger for MockMessenger {
    fn send(&self, msg: &str) {
        self.sent_messages.borrow_mut().push(msg.to_string());
    }
}

// Quota tracker using interior mutability
struct QuotaTracker<T: Messenger> {
    messenger: T,
    value: RefCell<usize>,
    max: usize,
}

impl<T: Messenger> QuotaTracker<T> {
    fn new(messenger: T, max: usize) -> Self {
        QuotaTracker {
            messenger,
            value: RefCell::new(0),
            max,
        }
    }
    
    fn set_value(&self, value: usize) {
        *self.value.borrow_mut() = value;
        
        let percentage = (value as f64 / self.max as f64) * 100.0;
        
        if percentage >= 100.0 {
            self.messenger.send("Error: You are over your quota!");
        } else if percentage >= 90.0 {
            self.messenger.send("Urgent warning: You've used up over 90% of your quota!");
        } else if percentage >= 75.0 {
            self.messenger.send("Warning: You've used up over 75% of your quota");
        }
    }
    
    fn current_value(&self) -> usize {
        *self.value.borrow()
    }
}

// Shared mutable state example
#[derive(Debug)]
struct Counter {
    value: RefCell<i32>,
    name: String,
}

impl Counter {
    fn new(name: String) -> Rc<Self> {
        Rc::new(Counter {
            value: RefCell::new(0),
            name,
        })
    }
    
    fn increment(&self) {
        *self.value.borrow_mut() += 1;
    }
    
    fn decrement(&self) {
        *self.value.borrow_mut() -= 1;
    }
    
    fn get(&self) -> i32 {
        *self.value.borrow()
    }
    
    fn display(&self) {
        println!("{}: {}", self.name, self.get());
    }
}

// Cell for Copy types
struct Statistics {
    count: Cell<u32>,
    sum: Cell<f64>,
}

impl Statistics {
    fn new() -> Self {
        Statistics {
            count: Cell::new(0),
            sum: Cell::new(0.0),
        }
    }
    
    fn add_value(&self, value: f64) {
        self.count.set(self.count.get() + 1);
        self.sum.set(self.sum.get() + value);
    }
    
    fn average(&self) -> f64 {
        if self.count.get() == 0 {
            0.0
        } else {
            self.sum.get() / self.count.get() as f64
        }
    }
    
    fn display(&self) {
        println!("Statistics: count={}, sum={:.2}, avg={:.2}", 
                self.count.get(), self.sum.get(), self.average());
    }
}

fn refcell_demo() {
    println!("\n=== RefCell<T> and Interior Mutability ===");
    
    // Mock messenger example
    let mock_messenger = MockMessenger::new();
    let quota_tracker = QuotaTracker::new(mock_messenger, 100);
    
    quota_tracker.set_value(50);
    quota_tracker.set_value(80);
    quota_tracker.set_value(95);
    quota_tracker.set_value(105);
    
    println!("Current quota value: {}", quota_tracker.current_value());
    println!("Messages sent:");
    for (i, message) in quota_tracker.messenger.messages().iter().enumerate() {
        println!("  {}: {}", i + 1, message);
    }
    
    // Shared counter example
    let counter = Counter::new("Shared Counter".to_string());
    
    // Multiple references to the same counter
    let counter_ref1 = Rc::clone(&counter);
    let counter_ref2 = Rc::clone(&counter);
    
    println!("\nShared counter operations:");
    counter.display();
    
    counter_ref1.increment();
    counter_ref1.increment();
    counter.display();
    
    counter_ref2.decrement();
    counter.display();
    
    counter_ref1.increment();
    counter_ref2.increment();
    counter.display();
    
    // Cell example for Copy types
    let stats = Statistics::new();
    
    println!("\nStatistics with Cell:");
    stats.display();
    
    stats.add_value(10.0);
    stats.add_value(20.0);
    stats.add_value(30.0);
    stats.display();
    
    stats.add_value(5.0);
    stats.add_value(15.0);
    stats.display();
    
    // Demonstrate runtime borrow checking
    println!("\nRuntime borrow checking:");
    let data = RefCell::new(vec![1, 2, 3, 4, 5]);
    
    // Multiple immutable borrows are allowed
    {
        let borrow1 = data.borrow();
        let borrow2 = data.borrow();
        println!("Multiple immutable borrows: {} and {}", borrow1.len(), borrow2.len());
    }
    
    // Mutable borrow
    {
        let mut borrow_mut = data.borrow_mut();
        borrow_mut.push(6);
        println!("After mutable borrow: {:?}", *borrow_mut);
    }
    
    // This would panic at runtime if uncommented:
    // let borrow1 = data.borrow();
    // let borrow_mut = data.borrow_mut(); // Panic: already borrowed
}

refcell_demo();

In [ ]:
// TODO: Complete the custom smart pointer implementation

use std::ops::Deref;
use std::rc::{Rc, Weak};
use std::cell::RefCell;

// Custom smart pointer with logging
struct LoggedBox<T> {
    data: Box<T>,
    id: u32,
}

impl<T> LoggedBox<T> {
    fn new(data: T, id: u32) -> Self {
        println!("Creating LoggedBox with ID: {}", id);
        LoggedBox {
            data: Box::new(data),
            id,
        }
    }
    
    fn id(&self) -> u32 {
        self.id
    }
}

impl<T> Deref for LoggedBox<T> {
    type Target = T;
    
    fn deref(&self) -> &Self::Target {
        &self.data
    }
}

impl<T> Drop for LoggedBox<T> {
    fn drop(&mut self) {
        println!("Dropping LoggedBox with ID: {}", self.id);
    }
}

// Resource manager using smart pointers
struct Resource {
    name: String,
    data: Vec<u8>,
}

impl Resource {
    fn new(name: String, size: usize) -> Self {
        println!("Allocating resource '{}' with {} bytes", name, size);
        Resource {
            name,
            data: vec![0; size],
        }
    }
    
    fn name(&self) -> &str {
        &self.name
    }
    
    fn size(&self) -> usize {
        self.data.len()
    }
}

impl Drop for Resource {
    fn drop(&mut self) {
        println!("Deallocating resource '{}'", self.name);
    }
}

// Resource pool using Rc and RefCell
struct ResourcePool {
    resources: RefCell<Vec<Rc<Resource>>>,
    next_id: RefCell<u32>,
}

impl ResourcePool {
    fn new() -> Self {
        ResourcePool {
            resources: RefCell::new(Vec::new()),
            next_id: RefCell::new(1),
        }
    }
    
    fn create_resource(&self, name: String, size: usize) -> Rc<Resource> {
        let id = *self.next_id.borrow();
        *self.next_id.borrow_mut() += 1;
        
        let resource_name = format!("{}-{}", name, id);
        let resource = Rc::new(Resource::new(resource_name, size));
        
        self.resources.borrow_mut().push(Rc::clone(&resource));
        resource
    }
    
    fn list_resources(&self) {
        let resources = self.resources.borrow();
        println!("Resource pool contains {} resources:", resources.len());
        for (i, resource) in resources.iter().enumerate() {
            println!("  {}: '{}' ({} bytes, {} refs)", 
                    i + 1, resource.name(), resource.size(), Rc::strong_count(resource));
        }
    }
    
    fn cleanup_unused(&self) {
        let mut resources = self.resources.borrow_mut();
        let initial_count = resources.len();
        
        // Remove resources that only have one reference (from the pool)
        resources.retain(|resource| Rc::strong_count(resource) > 1);
        
        let removed = initial_count - resources.len();
        if removed > 0 {
            println!("Cleaned up {} unused resources", removed);
        }
    }
}

// Cache using weak references to avoid cycles
struct Cache<K, V> {
    data: RefCell<std::collections::HashMap<K, Weak<V>>>,
}

impl<K, V> Cache<K, V>
where
    K: std::hash::Hash + Eq + Clone,
{
    fn new() -> Self {
        Cache {
            data: RefCell::new(std::collections::HashMap::new()),
        }
    }
    
    fn get(&self, key: &K) -> Option<Rc<V>> {
        let mut data = self.data.borrow_mut();
        
        if let Some(weak_ref) = data.get(key) {
            if let Some(strong_ref) = weak_ref.upgrade() {
                return Some(strong_ref);
            } else {
                // Remove expired weak reference
                data.remove(key);
            }
        }
        
        None
    }
    
    fn insert(&self, key: K, value: Rc<V>) {
        let weak_ref = Rc::downgrade(&value);
        self.data.borrow_mut().insert(key, weak_ref);
    }
    
    fn cleanup(&self) {
        let mut data = self.data.borrow_mut();
        let initial_size = data.len();
        
        data.retain(|_, weak_ref| weak_ref.strong_count() > 0);
        
        let removed = initial_size - data.len();
        if removed > 0 {
            println!("Cache cleanup: removed {} expired entries", removed);
        }
    }
    
    fn size(&self) -> usize {
        self.data.borrow().len()
    }
}

fn custom_smart_pointer_demo() {
    println!("\n=== Custom Smart Pointer Demo ===");
    
    // LoggedBox demonstration
    {
        let logged1 = LoggedBox::new("Hello".to_string(), 1);
        let logged2 = LoggedBox::new(42, 2);
        
        println!("LoggedBox 1 (ID {}): {}", logged1.id(), *logged1);
        println!("LoggedBox 2 (ID {}): {}", logged2.id(), *logged2);
        
        // Deref coercion works
        println!("String length: {}", logged1.len());
    } // LoggedBoxes are dropped here
    
    println!();
    
    // Resource pool demonstration
    let pool = ResourcePool::new();
    
    let resource1 = pool.create_resource("Database".to_string(), 1024);
    let resource2 = pool.create_resource("Cache".to_string(), 512);
    let resource3 = pool.create_resource("Logger".to_string(), 256);
    
    pool.list_resources();
    
    // Create additional references
    let db_ref = Rc::clone(&resource1);
    let cache_ref = Rc::clone(&resource2);
    
    println!("\nAfter creating additional references:");
    pool.list_resources();
    
    // Drop some references
    drop(resource3); // Only reference from pool remains
    drop(cache_ref); // cache_ref dropped, but resource2 still exists
    
    println!("\nAfter dropping some references:");
    pool.list_resources();
    
    pool.cleanup_unused();
    
    println!("\nAfter cleanup:");
    pool.list_resources();
    
    // Cache demonstration
    let cache: Cache<String, String> = Cache::new();
    
    let value1 = Rc::new("Cached Value 1".to_string());
    let value2 = Rc::new("Cached Value 2".to_string());
    
    cache.insert("key1".to_string(), Rc::clone(&value1));
    cache.insert("key2".to_string(), Rc::clone(&value2));
    
    println!("\nCache size: {}", cache.size());
    
    // Retrieve from cache
    if let Some(cached) = cache.get(&"key1".to_string()) {
        println!("Retrieved from cache: {}", *cached);
    }
    
    // Drop original references
    drop(value2);
    
    cache.cleanup();
    println!("Cache size after cleanup: {}", cache.size());
    
    // Try to get dropped value
    if cache.get(&"key2".to_string()).is_none() {
        println!("key2 is no longer in cache (expired)");
    }
    
    println!("\nSmart pointer demo complete!");
}

custom_smart_pointer_demo();

---

## ⚠️ Common Pitfalls: Smart Pointers

### 1. Reference Cycles with Rc<T>
```rust
use std::rc::Rc;
use std::cell::RefCell;

struct Node {
    next: Option<Rc<RefCell<Node>>>,
}

let a = Rc::new(RefCell::new(Node { next: None }));
let b = Rc::new(RefCell::new(Node { next: Some(Rc::clone(&a)) }));
a.borrow_mut().next = Some(Rc::clone(&b));  // ❌ Memory leak: cycle!
```
**Solution:** Use `Weak<T>` for back-references:
```rust
struct Node {
    next: Option<Rc<RefCell<Node>>>,
    prev: Option<Weak<RefCell<Node>>>,  // Weak reference
}
```

### 2. RefCell Borrow Violations at Runtime
```rust
use std::cell::RefCell;

let data = RefCell::new(5);
let borrow1 = data.borrow_mut();
let borrow2 = data.borrow_mut();  // ❌ Panic: already borrowed mutably
```
**Solution:** Ensure borrows don't overlap:
```rust
{
    let mut borrow1 = data.borrow_mut();
    *borrow1 += 1;
}  // borrow1 dropped here
let borrow2 = data.borrow_mut();  // ✅ OK now
```

### 3. Using Rc<T> Across Threads
```rust
use std::rc::Rc;
use std::thread;

let data = Rc::new(5);
thread::spawn(move || {
    println!("{}", data);  // ❌ Error: Rc is not Send
});
```
**Solution:** Use `Arc<T>` for thread-safe reference counting:
```rust
use std::sync::Arc;

let data = Arc::new(5);
let data_clone = Arc::clone(&data);
thread::spawn(move || {
    println!("{}", data_clone);  // ✅ Arc is Send + Sync
});
```

### 4. Forgetting to Upgrade Weak<T>
```rust
use std::rc::{Rc, Weak};

let strong = Rc::new(5);
let weak: Weak<i32> = Rc::downgrade(&strong);
drop(strong);

// ❌ Wrong: assuming weak is still valid
println!("{}", weak);  // Error: can't dereference Weak directly
```
**Solution:** Always upgrade and check:
```rust
match weak.upgrade() {
    Some(rc) => println!("{}", rc),
    None => println!("Value was dropped"),
}
```

### 5. Mutex Deadlocks
```rust
use std::sync::Mutex;

let data = Mutex::new(5);
let lock1 = data.lock().unwrap();
let lock2 = data.lock().unwrap();  // ❌ Deadlock: same thread tries to lock twice
```
**Solution:** Drop locks before re-acquiring or use scoping:
```rust
{
    let lock1 = data.lock().unwrap();
    // use lock1
}  // lock1 dropped
let lock2 = data.lock().unwrap();  // ✅ OK
```

### 6. Box<T> for Non-Recursive Types
```rust
// ⚠️ Unnecessary: i32 is already on stack
let boxed = Box::new(5);
```
**When to use Box:**
- ✅ Recursive types (linked lists, trees)
- ✅ Large types you want to move cheaply
- ✅ Trait objects: `Box<dyn Trait>`
- ❌ Simple stack values

### 7. Cloning Rc/Arc Instead of Using Clone
```rust
use std::rc::Rc;

let rc1 = Rc::new(5);
let rc2 = rc1.clone();  // ⚠️ Works but less clear
let rc3 = Rc::clone(&rc1);  // ✅ More explicit: cloning pointer, not data
```
**Best practice:** Use `Rc::clone(&rc)` to make it clear you're incrementing reference count

### 8. Interior Mutability Without Synchronization
```rust
use std::cell::RefCell;
use std::sync::Arc;

let data = Arc::new(RefCell::new(5));  // ❌ RefCell is not Sync
// Can't share across threads
```
**Solution:** Use `Mutex<T>` or `RwLock<T>` for thread-safe interior mutability:
```rust
use std::sync::{Arc, Mutex};

let data = Arc::new(Mutex::new(5));  // ✅ Thread-safe
```

**📚 Rust Book References:**
- [Chapter 15 - Smart Pointers](https://doc.rust-lang.org/book/ch15-00-smart-pointers.html)
- [Chapter 15.5 - RefCell and Interior Mutability](https://doc.rust-lang.org/book/ch15-05-interior-mutability.html)
- [Chapter 15.6 - Reference Cycles](https://doc.rust-lang.org/book/ch15-06-reference-cycles.html)